In [ ]:
import numpy as np
import json

class FlipDom:  
    def __init__(self):
        self.init_game()

    def init_game(self):
        self.board = np.zeros((8, 8), dtype=int)
        self.board[3, 3] = self.board[4, 4] = 1  
        self.board[3, 4] = self.board[4, 3] = -1  
        self.current_player = 1  

    def print_board(self):
        print("  0 1 2 3 4 5 6 7")
        for i, row in enumerate(self.board):
            print(i, end=" ")
            for cell in row:
                if cell == 1:
                    print("W", end=" ")
                elif cell == -1:
                    print("B", end=" ")
                else:
                    print(".", end=" ")
            print()
        print()

    def is_valid_move(self, row, col):
        if self.board[row, col] != 0:
            return False
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        opponent = -self.current_player

        for d in directions:
            r, c = row + d[0], col + d[1]
            found_opponent = False
            while 0 <= r < 8 and 0 <= c < 8 and self.board[r, c] == opponent:
                r, c = r + d[0], c + d[1]
                found_opponent = True
            if found_opponent and 0 <= r < 8 and 0 <= c < 8 and self.board[r, c] == self.current_player:
                return True
        return False

    def get_valid_moves(self):
        valid_moves = []
        for row in range(8):
            for col in range(8):
                if self.is_valid_move(row, col):
                    valid_moves.append((row, col))
        return valid_moves

    def make_move(self, row, col):
        if not self.is_valid_move(row, col):
            return False
        
        self.board[row, col] = self.current_player
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        opponent = -self.current_player

        for d in directions:
            r, c = row + d[0], col + d[1]
            flips = []
            while 0 <= r < 8 and 0 <= c < 8 and self.board[r, c] == opponent:
                flips.append((r, c))
                r, c = r + d[0], c + d[1]
            if flips and 0 <= r < 8 and 0 <= c < 8 and self.board[r, c] == self.current_player:
                for fr, fc in flips:
                    self.board[fr, fc] = self.current_player

        self.current_player = -self.current_player
        return True

    def is_game_over(self):
        return not self.get_valid_moves() and not self.get_valid_moves()

    def count_pieces(self):
        white_count = np.sum(self.board == 1)
        black_count = np.sum(self.board == -1)
        return white_count, black_count

    def evaluate_board(self):
        white_count, black_count = self.count_pieces()
        return black_count - white_count

    def undo_move(self, row, col):
        self.board[row, col] = 0
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        opponent = -self.current_player

        for d in directions:
            r, c = row + d[0], col + d[1]
            flips = []
            while 0 <= r < 8 and 0 <= c < 8 and self.board[r, c] == opponent:
                flips.append((r, c))
                r, c = r + d[0], c + d[1]
            if flips and 0 <= r < 8 and 0 <= c < 8 and self.board[r, c] == self.current_player:
                for fr, fc in flips:
                    self.board[fr, fc] = opponent

    def minimax(self, depth, alpha, beta, maximizing_player):
        if depth == 0 or self.is_game_over():
            return self.evaluate_board(), None

        valid_moves = self.get_valid_moves()
        best_move = None

        if maximizing_player:
            max_eval = float('-inf')
            for move in valid_moves:
                self.make_move(move[0], move[1])
                evaluation, _ = self.minimax(depth - 1, alpha, beta, False)
                self.undo_move(move[0], move[1])
                if evaluation > max_eval:
                    max_eval = evaluation
                    best_move = move
                alpha = max(alpha, evaluation)
                if beta <= alpha:
                    break
            return max_eval, best_move
        else:
            min_eval = float('inf')
            for move in valid_moves:
                self.make_move(move[0], move[1])
                evaluation, _ = self.minimax(depth - 1, alpha, beta, True)
                self.undo_move(move[0], move[1])
                if evaluation < min_eval:
                    min_eval = evaluation
                    best_move = move
                beta = min(beta, evaluation)
                if beta <= alpha:
                    break
            return min_eval, best_move

    def save_game(self, filename="flipdom_game.json"):  
        game_state = {
            "board": self.board.tolist(),
            "current_player": self.current_player
        }
        with open(filename, "w") as f:
            json.dump(game_state, f)
        print(f"Game saved to {filename}.")

    def load_game(self, filename="flipdom_game.json"):  
        with open(filename, "r") as f:
            game_state = json.load(f)
            self.board = np.array(game_state["board"])
            self.current_player = game_state["current_player"]
        print(f"Game loaded from {filename}.")

    def show_rules(self):
        rules = """
        FlipDom Game Rules:
        1. The game is played on an 8x8 board.
        2. Players take turns placing pieces on the board.
        3. A valid move will flip the opponent's pieces in one or more directions.
        4. The game ends when no more valid moves are available for both players.
        5. The player with the most pieces at the end of the game wins.
        """
        print(rules)

    def play(self):
        self.show_rules() 
        while not self.is_game_over():
            self.print_board()
            valid_moves = self.get_valid_moves()

            if valid_moves:
                print(f"Current player: {'White' if self.current_player == 1 else 'Black'}")
                print("Valid moves:", valid_moves)

                if self.current_player == 1:  
                    while True:
                        try:
                            row = int(input("Enter row (0-7): ").strip())
                            if row < 0 or row > 7:
                                print("Row must be between 0 and 7. Try again.")
                                continue
                            col = int(input("Enter column (0-7): ").strip())
                            if col < 0 or col > 7:
                                print("Column must be between 0 and 7. Try again.")
                                continue
                            break  
                        except ValueError:
                            print("Invalid input, please try again.")

                    if (row, col) in valid_moves:
                        self.make_move(row, col)
                    else:
                        print("Invalid move, please try again.")
                else: 
                    depth = 3
                    _, move = self.minimax(depth, float('-inf'), float('inf'), True)
                    if move:
                        print(f"AI chooses move: {move}")
                        self.make_move(move[0], move[1])
            else:
                print(f"No valid moves for player {'White' if self.current_player == 1 else 'Black'}")
                self.current_player = -self.current_player  

        white_count, black_count = self.count_pieces()
        print("Game over!")
        print(f"White pieces: {white_count}, Black pieces: {black_count}")
        if white_count > black_count:
            print("White wins!")
        elif black_count > white_count:
            print("Black wins!")
        else:
            print("It's a draw!")

        restart = input("Do you want to play again? (y/n): ").strip().lower()
        if restart == 'y':
            self.init_game()
            self.play()
        else:
            print("Thank you for playing!")

# Run the game
game = FlipDom()  
game.play()



        FlipDom Game Rules:
        1. The game is played on an 8x8 board.
        2. Players take turns placing pieces on the board.
        3. A valid move will flip the opponent's pieces in one or more directions.
        4. The game ends when no more valid moves are available for both players.
        5. The player with the most pieces at the end of the game wins.
        
  0 1 2 3 4 5 6 7
0 . . . . . . . . 
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . W B . . . 
4 . . . B W . . . 
5 . . . . . . . . 
6 . . . . . . . . 
7 . . . . . . . . 

Current player: White
Valid moves: [(2, 4), (3, 5), (4, 2), (5, 3)]


Enter row (0-7):  3
Enter column (0-7):  5


  0 1 2 3 4 5 6 7
0 . . . . . . . . 
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . W W W . . 
4 . . . B W . . . 
5 . . . . . . . . 
6 . . . . . . . . 
7 . . . . . . . . 

Current player: Black
Valid moves: [(2, 3), (2, 5), (4, 5)]
AI chooses move: (2, 5)
  0 1 2 3 4 5 6 7
0 . . . . . . . . 
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . B W W . . 
4 . . . W B . . . 
5 . . . . . . . . 
6 . . . . . . . . 
7 . . . . . . . . 

Current player: White
Valid moves: [(2, 3), (3, 2), (4, 5), (5, 3), (5, 4)]


Enter row (0-7):  5
Enter column (0-7):  4


  0 1 2 3 4 5 6 7
0 . . . . . . . . 
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . B W W . . 
4 . . . W W . . . 
5 . . . . W . . . 
6 . . . . . . . . 
7 . . . . . . . . 

Current player: Black
Valid moves: [(3, 6), (5, 3), (5, 5)]
AI chooses move: (3, 6)
Game over!
White pieces: 6, Black pieces: 0
White wins!
